# SQLLite

SQLite es probablemente la base de datos más sencilla para conectarse con una aplicación de Python, ya que no necesita instalar ningún módulo SQL de Python externo para hacerlo. De manera predeterminada, su instalación de Python contiene una biblioteca SQL de Python llamada sqlite3 que puede usar para interactuar con una base de datos SQLite.

Además, las bases de datos SQLite no tienen servidor y son independientes, ya que leen y escriben datos en un archivo. Esto significa que, a diferencia de MySQL y PostgreSQL, ¡ni siquiera necesita instalar y ejecutar un servidor SQLite para realizar operaciones de base de datos!

Miremos como conectarnos a sqlite3 con una base de datos en Python:

In [32]:
import sqlite3
from sqlite3 import Error # Importar librerias
def create_connection(path): # Definir funcion crear conexion
    connection = None
    try:
        connection = sqlite3.connect(path) # uso de connect() que recibe una ruta a la base de datos SQLLite (puede estar en la ram o no)
        print("Conexion a SQLite DB exitosa") # Si la conexion es exitosa se imprime
    except Error as e: # Si existe algun error
        print(f"El error '{e}' ha ocurrido") # mostrar el error
    return connection

# Conexiones
* ```sqlite3.connect(ruta)``` devuelve un objeto de conexión, que a su vez es devuelto por ```create_connection()```. 
* Este objeto de conexión se puede utilizar para ejecutar consultas en una base de datos SQLite. 
* El siguiente script crea una conexión a la base de datos SQLite:

In [33]:
connection = create_connection("david.sqlite")

Conexion a SQLite DB exitosa


# Crear tablas SQL Lite

* Para ejecutar consultas en SQLite, debemos usar ```cursor.execute()```. 
* Definiremos una función llamada ```execute_query()``` que usa este método. 
* Su función aceptará el objeto de conexión y una cadena de consulta, que pasará a ```cursor.execute()```.
* ```.execute()``` puede ejecutar cualquier consulta que se le pase en forma de string. 

In [34]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query ejecutado satisfactoriamente")
    except Error as e:
        print(f"El error '{e}' ha ocurrido")

In [35]:
# Armamos el QUERY
crear_tabla_usuarios = """
CREATE TABLE IF NOT EXISTS users (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  name TEXT NOT NULL,
  age INTEGER,
  gender TEXT,
  nationality TEXT
);
"""

In [36]:
# Ejecuto el query
execute_query(connection, crear_tabla_usuarios) 

Query ejecutado satisfactoriamente


In [37]:
# Creo query para generar segunda tabla
crear_tabla_posts = """
CREATE TABLE IF NOT EXISTS posts(
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  title TEXT NOT NULL, 
  description TEXT NOT NULL, 
  user_id INTEGER NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id)
);
"""
# Dado que existe una relación de uno a muchos entre los usuarios y las publicaciones, 
# veran una clave externa user_id en la tabla de publicaciones que hace referencia 
# a la columna de identificación en la tabla de usuarios. 

In [38]:
# Ejecuto el query
execute_query(connection, crear_tabla_posts)

Query ejecutado satisfactoriamente


In [39]:
# Dos tablas mas: 
crear_tabla_comentarios = """
CREATE TABLE IF NOT EXISTS comments (
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  text TEXT NOT NULL, 
  user_id INTEGER NOT NULL, 
  post_id INTEGER NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id) FOREIGN KEY (post_id) REFERENCES posts (id)
);
"""

crear_tabla_likes = """
CREATE TABLE IF NOT EXISTS likes (
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  user_id INTEGER NOT NULL, 
  post_id integer NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id) FOREIGN KEY (post_id) REFERENCES posts (id)
);
"""

execute_query(connection, crear_tabla_comentarios)  
execute_query(connection, crear_tabla_likes)     

Query ejecutado satisfactoriamente
Query ejecutado satisfactoriamente


In [40]:
# Llenamos las tablas 
crear_usuarios = """
INSERT INTO
  users (name, age, gender, nationality)
VALUES
  ('James', 25, 'hombre', 'USA'),
  ('Leila', 32, 'mujer', 'France'),
  ('Brigitte', 35, 'mujer', 'England'),
  ('Mike', 40, 'hombre', 'Denmark'),
  ('Elizabeth', 21, 'mujer', 'Canada');
"""
execute_query(connection, crear_usuarios)    

Query ejecutado satisfactoriamente


In [41]:
crear_posts = """
INSERT INTO
  posts (title, description, user_id)
VALUES
  ("Feliz", "Me siento feliz hoy", 1),
  ("Caliente", "El clima esta caliente hoy", 2),
  ("Ayuda", "Necesito ayuda en esto", 2),
  ("Buenas noticias", "Me casare pronto", 1),
  ("Juego interesante", "Fue genial jugar al tenis", 5),
  ("Fiesta", "Alguno quiere venir a esta fiesta hoy?", 3);
"""
execute_query(connection, crear_posts)  

Query ejecutado satisfactoriamente


In [42]:
crear_comentarios = """
INSERT INTO
  comments (text, user_id, post_id)
VALUES
  ('Cuenta conmigo', 1, 6),
  ('Que tipo de ayuda?', 5, 3),
  ('Felicitaciones', 2, 4),
  ('Estuve jugando con Rafael', 4, 5),
  ('Te ayudo con tu tesis?', 2, 3),
  ('Muchas felicitaciones', 5, 4);
"""

crear_likes = """
INSERT INTO
  likes (user_id, post_id)
VALUES
  (1, 6),
  (2, 3),
  (1, 5),
  (5, 4),
  (2, 4),
  (4, 2),
  (3, 6);
"""

execute_query(connection, crear_comentarios)
execute_query(connection, crear_likes)  

Query ejecutado satisfactoriamente
Query ejecutado satisfactoriamente


# Selección de data
Para trabajar en los análisis vistos hasta el momento, debemos conectarnos a la base de datos y obtener la información que necesitamos. 

In [43]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"Error '{e}' ha ocurrido")

In [44]:
elegir_usuarios = "SELECT * from users"
users = execute_read_query(connection, elegir_usuarios)

users

[(1, 'James', 25, 'hombre', 'USA'),
 (2, 'Leila', 32, 'mujer', 'France'),
 (3, 'Brigitte', 35, 'mujer', 'England'),
 (4, 'Mike', 40, 'hombre', 'Denmark'),
 (5, 'Elizabeth', 21, 'mujer', 'Canada'),
 (6, 'James', 25, 'hombre', 'USA'),
 (7, 'Leila', 32, 'mujer', 'France'),
 (8, 'Brigitte', 35, 'mujer', 'England'),
 (9, 'Mike', 40, 'hombre', 'Denmark'),
 (10, 'Elizabeth', 21, 'mujer', 'Canada')]

In [45]:
elegir_posts = "SELECT * FROM posts"
posts = execute_read_query(connection, elegir_posts)

posts

[(1, 'Feliz', 'Me siento feliz hoy', 1),
 (2, 'Caliente', 'El clima esta caliente hoy', 2),
 (3, 'Ayuda', 'Necesito ayuda en esto', 2),
 (4, 'Buenas noticias', 'Me casare pronto', 1),
 (5, 'Juego interesante', 'Fue genial jugar al tenis', 5),
 (6, 'Fiesta', 'Alguno quiere venir a esta fiesta hoy?', 3),
 (7, 'Feliz', 'Me siento feliz hoy', 1),
 (8, 'Caliente', 'El clima esta caliente hoy', 2),
 (9, 'Ayuda', 'Necesito ayuda en esto', 2),
 (10, 'Buenas noticias', 'Me casare pronto', 1),
 (11, 'Juego interesante', 'Fue genial jugar al tenis', 5),
 (12, 'Fiesta', 'Alguno quiere venir a esta fiesta hoy?', 3)]

In [46]:
joins_usuarios_posts = """
SELECT
  users.id,
  users.name,
  posts.description
FROM
  posts
  INNER JOIN users ON users.id = posts.user_id
"""

usuarios_posts = execute_read_query(connection, joins_usuarios_posts)
usuarios_posts

[(1, 'James', 'Me siento feliz hoy'),
 (2, 'Leila', 'El clima esta caliente hoy'),
 (2, 'Leila', 'Necesito ayuda en esto'),
 (1, 'James', 'Me casare pronto'),
 (5, 'Elizabeth', 'Fue genial jugar al tenis'),
 (3, 'Brigitte', 'Alguno quiere venir a esta fiesta hoy?'),
 (1, 'James', 'Me siento feliz hoy'),
 (2, 'Leila', 'El clima esta caliente hoy'),
 (2, 'Leila', 'Necesito ayuda en esto'),
 (1, 'James', 'Me casare pronto'),
 (5, 'Elizabeth', 'Fue genial jugar al tenis'),
 (3, 'Brigitte', 'Alguno quiere venir a esta fiesta hoy?')]

In [47]:
# Para obtener nombre de columnas: 
cursor = connection.cursor()
cursor.execute(joins_usuarios_posts)
cursor.fetchall()

columnas = [description[0] for description in cursor.description]

In [48]:
columnas

['id', 'name', 'description']

In [49]:
import pandas as pd
usuarios_posts_df = pd.DataFrame(usuarios_posts,
                                 columns = columnas)
usuarios_posts_df

,id,name,description
0,1,James,Me siento feliz hoy
1,2,Leila,El clima esta caliente hoy
2,2,Leila,Necesito ayuda en esto
3,1,James,Me casare pronto
4,5,Elizabeth,Fue genial jugar al tenis
5,3,Brigitte,Alguno quiere venir a esta fiesta hoy?
6,1,James,Me siento feliz hoy
7,2,Leila,El clima esta caliente hoy
8,2,Leila,Necesito ayuda en esto
9,1,James,Me casare pronto


In [51]:
usuarios_posts_df['id'].mean()

2.3333333333333335